In [25]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [115]:
data1 = np.genfromtxt('data1.dat', delimiter='  ')
data1 = data1.transpose()

print data1

[[-1.63266415e+00 -1.12359208e+00  2.65037590e-01 -3.03251515e-01
  -7.81113242e-01  2.96823204e-01 -2.37963194e-01  4.60391537e-01
   4.78811835e-01  1.61110050e+00  2.21650744e+00  1.19354135e+00
  -7.38588492e-01  7.68281412e-01  8.81442728e-01 -4.15594033e-01
   1.96303401e-01 -1.15142320e+00 -1.57764442e-01 -1.02848856e-01
   8.95039117e-01  5.14555754e-01 -1.43027775e-01  2.04448916e+00
   9.62663709e-01 -5.68839104e-01 -2.20651581e-01  3.76690886e-01
   2.43335101e-01  1.27122875e-01 -2.57705425e+00  1.32175605e+00
   1.79206597e+00 -5.68064660e-01 -5.72602654e-01 -2.46168200e-04
  -1.36627890e+00  7.95196719e-01 -8.37574386e-01 -3.39800500e-01
  -6.86637455e-01 -8.27478500e-01 -3.33782808e-01 -7.64036114e-02
  -1.21107124e+00 -7.75199654e-01 -1.57450427e+00 -1.35833493e+00
   2.60362965e+00  4.82439540e-01 -6.77531629e-01 -5.48726646e-02
   2.31493114e+00  8.89522813e-01  1.48678174e+00  1.32491721e+00
   1.00357132e-01 -7.82474319e-01  2.06397499e-02 -1.26864382e+00
  -5.78534

In [111]:
#part a - estimate using OLS
y = data1[0]
x = sm.add_constant(data1[1])
part_a = sm.OLS(y,x).fit()
print part_a.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     1979.
Date:                Thu, 13 Sep 2018   Prob (F-statistic):          1.33e-175
Time:                        17:02:05   Log-Likelihood:                -362.72
No. Observations:                 500   AIC:                             729.4
Df Residuals:                     498   BIC:                             737.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0055      0.022      0.245      0.8

In [ ]:
#part b - MLE

class part_b(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        #q = 2 * endog - 1
        #return stats.norm.logcdf(q*np.dot(exog, params)).sum()




In [114]:
#part c - GMM
#https://github.com/josef-pkt/misc/blob/master/notebooks/ex_gmm_gamma.ipynb
#https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.GMM.html#statsmodels.sandbox.regression.gmm.GMM
#https://gist.github.com/josef-pkt/6895915


class part_c(GMM):
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', 2)
        kwds.setdefault('k_params',2)
        super(part_c, self).__init__(*args, **kwds)

    def momcond(self, params):
        t1,t2 = params   
        #unwrap endog
        endog = self.endog
        exog = self.exog.squeeze()
        eps = endog - t1 - t2*exog
        g = np.column_stack( (eps, eps*exog ))
        return g 

    
model_c = part_c(data1[0],data1[1], None)
beta0 = np.array([1, 1])
resc = model_c.fit(beta0, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(resc.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 40
         Function evaluations: 80
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 14
         Function evaluations: 28
                                part_c Results                                
Dep. Variable:                      y   Hansen J:                    1.730e-06
Model:                         part_c   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Thu, 13 Sep 2018                                         
Time:                        17:03:40                                         
No. Observations:                 500                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0          